In [1]:
import os
import sys
from typing import (
    List,
    Dict,
    Tuple,
    Union,
)
import numpy as np
import matplotlib.pyplot as plt
import cv2 as cv

%matplotlib inline

In [2]:
sys.path.append("../../../lib")

In [53]:
%load_ext autoreload
%autoreload 2

from util_opencv.image import (
    validate_image,
    get_image,
    save_image,
    get_image_dimensions,
    resize_image,
    show_image,
    show_image_opencv,
    resize_and_save_images,
    rotate,
)
from util_numpy import (
    get_cosine_similarity,
)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Resize images

In [43]:
resize_and_save_images(
    path_to_source="../data/original",
    pattern="*.jpg",
    height=224,
    width=224,
    path_to_destination="../data/resized"
)

ERROR:util_opencv.image:get_image(): OpenCV has no image reader for the image [../data/original/03055.jpg].
INFO:util_opencv.image:resize_and_save_images(): cannot handle and skip [../data/original/03055.jpg]...
ERROR:util_opencv.image:get_image(): OpenCV has no image reader for the image [../data/original/04035.jpg].
INFO:util_opencv.image:resize_and_save_images(): cannot handle and skip [../data/original/04035.jpg]...
ERROR:util_opencv.image:get_image(): OpenCV has no image reader for the image [../data/original/01218.jpg].
INFO:util_opencv.image:resize_and_save_images(): cannot handle and skip [../data/original/01218.jpg]...
ERROR:util_opencv.image:get_image(): OpenCV has no image reader for the image [../data/original/04036.jpg].
INFO:util_opencv.image:resize_and_save_images(): cannot handle and skip [../data/original/04036.jpg]...


---

In [10]:
from keras.models import (
    Model, 
    Sequential
)

from tensorflow.keras.applications.resnet50 import (
    ResNet50,
    preprocess_input, 
    decode_predictions
)
from keras.preprocessing import image
import tensorflow as tf
import graphviz
import pydot

# Classification by ResNet50

* [ResNet50](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/ResNet50)
* [tf.keras.applications.resnet50.preprocess_input](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/preprocess_input)

> ```
> tf.keras.applications.resnet50.preprocess_input(
>    x, data_format=None
>)
> """
> Args:
>     x:  
>         A floating point numpy.array or a tf.Tensor, 3D or 4D with 3 color channels, 
>         with values in the range [0, 255]. The preprocessed data are written over 
>         the input data if the data types are compatible. To avoid this behaviour, 
>         numpy.copy(x) can be used.  
> Returns: Preprocessed numpy.array or a tf.Tensor with type float32.
> """
> ```

* [tf.keras.applications.resnet50.decode_predictions](https://www.tensorflow.org/api_docs/python/tf/keras/applications/resnet50/decode_predictions)

> Returns: A list of lists of top class prediction tuples ```(class_name, class_description, score)```. One list of tuples per sample in batch input.


## Examples

* [Usage examples for image classification models - Classify ImageNet classes with ResNet50](https://keras.io/api/applications/#usage-examples-for-image-classification-models)

> ```
> from tensorflow.keras.applications.resnet50 import ResNet50
> from tensorflow.keras.preprocessing import image
> from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
> import numpy as np
> 
> model = ResNet50(weights='imagenet')
> 
> img_path = 'elephant.jpg'
> img = image.load_img(img_path, target_size=(224, 224))
> x = image.img_to_array(img)
> x = np.expand_dims(x, axis=0)
> x = preprocess_input(x)
> 
> preds = model.predict(x)
> # decode the results into a list of tuples (class, description, probability)
> # (one such list for each sample in the batch)
> print('Predicted:', decode_predictions(preds, top=3)[0])
> # Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]
> ```

In [11]:
def convert_image_array_for_resnet(image: np.ndarray):
    """Transform np array of an image to match the input format of Keras ResNet.
    """
    # TF/Keras is batch based. Add the dimension for batch for an image of shape (244, 244, 3)
    validate_image(image)
    x = np.expand_dims(image,axis=0)
    
    # Make sure the data is compatible with TF (e.g. 32 bit float)
    return preprocess_input(x)

In [12]:
model = ResNet50(weights='imagenet')

In [13]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                           

                                                                                                  
 conv2_block3_1_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv2_block3_2_conv (Conv2D)   (None, 56, 56, 64)   36928       ['conv2_block3_1_relu[0][0]']    
                                                                                                  
 conv2_block3_2_bn (BatchNormal  (None, 56, 56, 64)  256         ['conv2_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv2_block3_2_relu (Activatio  (None, 56, 56, 64)  0           ['conv2_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv3_block3_1_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block3_2_conv (Conv2D)   (None, 28, 28, 128)  147584      ['conv3_block3_1_relu[0][0]']    
                                                                                                  
 conv3_block3_2_bn (BatchNormal  (None, 28, 28, 128)  512        ['conv3_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv3_block3_2_relu (Activatio  (None, 28, 28, 128)  0          ['conv3_block3_2_bn[0][0]']      
 n)       

                                                                                                  
 conv4_block2_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block2_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block2_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block2_1_relu[0][0]']    
                                                                                                  
 conv4_block2_2_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block2_2_conv[0][0]']    
 ization) 

 conv4_block5_1_conv (Conv2D)   (None, 14, 14, 256)  262400      ['conv4_block4_out[0][0]']       
                                                                                                  
 conv4_block5_1_bn (BatchNormal  (None, 14, 14, 256)  1024       ['conv4_block5_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block5_1_relu (Activatio  (None, 14, 14, 256)  0          ['conv4_block5_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv4_block5_2_conv (Conv2D)   (None, 14, 14, 256)  590080      ['conv4_block5_1_relu[0][0]']    
                                                                                                  
 conv4_blo

                                                                  'conv5_block1_3_bn[0][0]']      
                                                                                                  
 conv5_block1_out (Activation)  (None, 7, 7, 2048)   0           ['conv5_block1_add[0][0]']       
                                                                                                  
 conv5_block2_1_conv (Conv2D)   (None, 7, 7, 512)    1049088     ['conv5_block1_out[0][0]']       
                                                                                                  
 conv5_block2_1_bn (BatchNormal  (None, 7, 7, 512)   2048        ['conv5_block2_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_1_relu (Activatio  (None, 7, 7, 512)   0           ['conv5_block2_1_bn[0][0]']      
 n)       

In [60]:
image=get_image(path="../data/resized/03809.jpg")
x: np.ndarray = convert_image_array_for_resnet(image)

In [61]:
predictions = model.predict(x)
for pred in decode_predictions(predictions, top=3)[0]:  # [0] for the first image as TF handles in batch by default
    print(pred)

1/1 [==============================] - 0s 48ms/step
('n01833805', 'hummingbird', 0.15104976)
('n03485794', 'handkerchief', 0.058680423)
('n02206856', 'bee', 0.052906573)


---

# Feature Extraction from ResNet50 

In [50]:
feature_extractor = Model(
    inputs=model.input, 
    outputs=model.get_layer("avg_pool").output
)

In [62]:
embedding_03809 = (feature_extractor(x)[0]).numpy()

In [52]:
embedding_02322

array([0.17491272, 0.00635768, 0.1570035 , ..., 0.27677226, 0.5323709 ,
       0.0443731 ], dtype=float32)

In [58]:
embedding_02320

array([0.0503598 , 0.0029425 , 0.2126748 , ..., 1.510168  , 0.0364648 ,
       0.03879821], dtype=float32)

In [59]:
get_cosine_similarity(x=embedding_02320, y=embedding_02322)

array([[0.6815219]], dtype=float32)

In [63]:
embedding_03809

array([3.4748464e+00, 6.0930056e-04, 7.4448973e-01, ..., 4.3872476e-01,
       7.8435190e-02, 7.3485720e-01], dtype=float32)

In [64]:
get_cosine_similarity(x=embedding_02320, y=embedding_03809)

array([[0.19784549]], dtype=float32)